In [ ]:
!pip install pydub #install necessary directories
!apt install ffmpeg
!pip install spleeter
!pip install librosa
!pip install keras
!pip install numpy

In [ ]:
from google.colab import drive #mount to google drive to export files
drive.mount('/content/drive', force_remount=True)
from IPython.display import Audio
from pydub import AudioSegment
import librosa
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM
import numpy

In [ ]:
!wget https://github.com/manntalati/musicproj/raw/main/songs/Numb.mp3 #receive music files from github
!wget https://github.com/manntalati/musicproj/raw/main/songs/DigitalLove.mp3
!wget https://github.com/manntalati/musicproj/raw/main/songs/Faint.mp3
!wget https://github.com/manntalati/musicproj/raw/main/songs/Rock.mp3
!wget https://github.com/manntalati/musicproj/raw/main/songs/Lost.mp3

In [ ]:
!spleeter separate -o output -p spleeter:4stems Numb.mp3 #split the mp3 files to find the drums.wav using spleeter
!spleeter separate -o output -p spleeter:4stems DigitalLove.mp3
!spleeter separate -o output -p spleeter:4stems Lost.mp3
!spleeter separate -o output -p spleeter:4stems Faint.mp3
!spleeter separate -o output -p spleeter:4stems Rock.mp3

In [ ]:
sound1 = AudioSegment.from_wav('output/Numb/drums.wav') #export the files to google drive under test data specifically for drums audio only
sound1.export('/content/drive/MyDrive/musicproj/TrainData/NumbDrums.mp3', format='mp3')
sound2 = AudioSegment.from_wav('output/Lost/drums.wav')
sound2.export('/content/drive/MyDrive/musicproj/TrainData/LostDrums.mp3', format='mp3')
sound3 = AudioSegment.from_wav('output/Rock/drums.wav')
sound3.export('/content/drive/MyDrive/musicproj/TrainData/RockDrums.mp3', format='mp3')
sound4 = AudioSegment.from_wav('output/DigitalLove/drums.wav')
sound4.export('/content/drive/MyDrive/musicproj/TrainData/DigitalLoveDrums.mp3', format='mp3')
sound5 = AudioSegment.from_wav('output/Faint/drums.wav')
sound5.export('/content/drive/MyDrive/musicproj/TrainData/FaintDrums.mp3', format='mp3')

In [7]:
def make_pieces(song): #cut the song up into pieces by going from start to every 50,000 from start
  start = 0
  end = len(song)
  
  pieces = []

  while start < end - 50000:
    pieces.append(song[start:start+50000])

    start += 50000

  return pieces #return the array of all of the pieces of the song

In [8]:
def specto_song(song_path): #turn the different pieces of song into a spectrogram
  song_array = []
  y,sr = librosa.load(song_path) #load through the librosa module
  pieces = make_pieces(y) #utilize the make_pieces function above that splits the song
  for piece in pieces:
    spectrogram = librosa.feature.melspectrogram(y=piece) #convert each piece into a spectrogram
    song_array.append(spectrogram) #add each piece into a separate array to return
  return song_array

In [9]:
songs = ['Numb', 'Lost', 'Rock', 'DigitalLove', 'Faint'] #utilize array to create a separate array with all of the songs paths
songs_path = []
for i in range(0, 5):
  songs_path.append('/content/drive/MyDrive/musicproj/TrainData/' + songs[i] + 'Drums.mp3')
  i+=1

In [21]:
model_input = []
for song in songs_path:
  y, sr = librosa.load(song) #use librosa load for each drum song
  model_input.append(specto_song(song)) #print(specto_song(song)) #print the array out that is generated for each song through the specto_song and make_pieces functions

In [88]:
x = (model_input[0])[0]
y = (model_input[0])[1]
x = numpy.array(X)
y = numpy.array(y)
x = x.reshape(1, 112, 112)
y = y.reshape(1, 112, 112)

In [87]:
model = Sequential()
model.add(LSTM(8, input_shape=(112, 112), return_sequences=False))
model.add(Dense(2, kernel_initializer='normal',activation='linear'))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))